## Scraping St. Louis Weather Data

## Problem

Data exists in a variety of forms from a variety of places.  Sometimes data will need to be pulled from raw HTML to get at it.  We want to simulate an IoT device by grabbing data from the weather station at the St. Louis Science Center, which at the moment, we have no other way to get at this data.

### Solution

* grab HTML at the end of the site: [http://agebb.missouri.edu/weather/realtime/st_louis_science_center.php](http://agebb.missouri.edu/weather/realtime/st_louis_science_center.php)
* parse the data for temperature and humdity
* store the data on the IoT block chain

### Code

In [ ]:
# uncomment and run necessary libraries if not already installed
# pip install html5lib
# pip install bs4

In [1]:
# importing necessary libraries

import datetime
import requests
import csv
import pandas as pd
from bs4 import BeautifulSoup
import json

In [2]:
# Accessing the html content from webpage
url = "http://agebb.missouri.edu/weather/realtime/st_louis_science_center.php"
results = requests.get(url)
rdata = results.content
soup = BeautifulSoup(rdata, 'html5lib')
timestamp = datetime.datetime.now().isoformat()

In [3]:
# Accessing content in tags
# Desired feature group is in a collapsible div with'id':'current-collapsible'
# features class on webpage is "ui-body" while value class is "ui-bar"

table = soup.find('div', {'id':'current-collapsible'})  # accessing the collapsible section

data_file = []
# looping through the first 3 features (Temperature, dewpoint and humidity) in the table
for r in table.findAll('div', attrs = {'class':"ui-grid-a"})[:3]:
    stldata = {}
    stldata['feature'] = r.find('div', {'class':"ui-body"}).string
    stldata['value'] = r.find('div', {'class':"ui-bar"}).string
    data_file.append(stldata)

#
# payload will be of the form  
# {
#     "temp": 22.1,
#     "rh": 55,
#     "timestamp": "YYYY-MM-DDTHH:MM:SSZ"
# }    
#

payload = {}
for data in data_file:
    if data['feature'][:4] == 'Temp':
        payload['temp'] = data['value'].replace("°F", "")
    if data['feature'][:4] == 'Humi':
        payload['rh'] = data['value'].replace("%", "")
    payload['timestamp'] = timestamp
    payload['stid'] = "stl-science-center"


Show the payload

In [4]:
payload

{'rh': '19',
 'stid': 'stl-science-center',
 'temp': '75.1',
 'timestamp': '2021-11-08T21:40:27.730860'}

Put the payload on the blockchain

In [16]:
block_chain_server = "34.69.195.189:5000"
#block_chain_server = "localhost:5000"

headers = {
    'Content-type': 'application/json'
}

r = requests.post(f"http://{block_chain_server}/add", data=json.dumps(payload), headers=headers)
if r.status_code == 200:
    resp = r.json()
    print(resp)
else:
    print(f"[info] unable to communicate with server or add data block : {r.status_code}")

{'hash': '391c43daec4a3b2dae141bbd05d6f977696e14236ba484d4cf2a1cf8084bb7cf', 'message': 'block added', 'status': 'ok'}


Test the hash data block:

In [18]:
r = requests.get(f"http://{block_chain_server}/{resp['hash']}", data=payload, headers=headers)
if r.status_code == 200:
    resp=r.json()

In [20]:
resp

['391c43daec4a3b2dae141bbd05d6f977696e14236ba484d4cf2a1cf8084bb7cf',
 'f2073d3e53e47e7cde7017d3318943f1da7401ac0b6106aff84183ce77ad290d',
 ['{"temp": "75.1", "timestamp": "2021-11-08T21:40:27.730860", "stid": "stl-science-center", "rh": "19"}'],
 '1636407856.1470275']

In [21]:
resp[2]

['{"temp": "75.1", "timestamp": "2021-11-08T21:40:27.730860", "stid": "stl-science-center", "rh": "19"}']

In [22]:
r = requests.get(f"http://{block_chain_server}/chain", data=json.dumps(payload), headers=headers)
if r.status_code == 200:
    print(r.json())

['b63716b32cd2e72dfdf86aa01e4a5a44a616d28cb996204eee4a8e718379f609', '9d70eaf8bf812b15e6f68c11500459a76adee770e295887727ff8c7d1c96996a', '83307a54da1f8111282451290763574d6db65bd349782041d65e055271e93a14', '8fc08df94016a8011d5f58f7e4171af42593ad8aa42d00d38e53dc8076723f0a', '414cbf97d5b17b87726549278fd74acfc00adc0da507d4531ae45259b9b2d6d3', 'ab70596734a78f3b4e500023d7949167de6c82d774a49768e721c357f12106e7', 'c9e46b0088384849be423a6e74cbc232d59532528fd829db2cb156e4b07c351e', '6708ac74b10030e57b6cf3cb427046ded5e6911c3425f894c7fb59ebfc46a875', '164f4badcafcae8ab3cf5bd6aef393ba1ae133075b1f53e1158f2bbb4a1cb18e', 'fc2069c47bcff62bcc3bd305abda52c47b4e7776d14cf54fe3515517368f79d0', '9b05204a06e66d8f92a30319249f6ad5d64494a04d22cdde7d55f1f85c2b8b91', 'bfb6977b1f3cdffff42b55a0fd8e2797db8d8d3e23034f164897e424a500ecb9', 'bb2f72d9dd3b1a436dc5d5a3f19a8e5595b5b05cb10b2cd463fc06602e3b4c48', '30dc4fa391cfad5a5f080486937dbf6047d93ab57a3688678b7254b4a7ecec8e', '0d7c1ee82b98f1e2fd71c28ef38bd228cffe0898951cc3